In [ ]:
import numpy as np 

import sys
import matplotlib
#matplotlib.use('Agg')
sys.path.append('/global/u1/j/jelvinpo/DES/lss_sys/')
import lsssys
import twopoint
import fitsio as fio
import healpy as hp 


In [4]:
#paths and config

mock_dir = '/global/project/projectdirs/des/monroy/lognormal_mocks/contaminated_mocks/redmagic_v0.5.1_gold_2.2.1_newzbinning_jointmask_contaminated_lognormal_mocks/contaminated_lognormal_mocks_2.0sig/'
mock_file_temp = mock_dir + 'y3_redmagic_v0.5.1_newzbinning_contaminated_2.0sig_nside512_mock{imock}.fits.gz'

f_map_dir = '/project/projectdirs/des/nweaverd/y3_enet/systematics_analysis/output/fracdet_thresh_0d1/mocks/fest_maps/'
f_map_temp = f_map_dir+'{label}_Fest_map_Nbase108_512_izbin{ibin}_minfrac0.1_mock{imock}.fits.gz'

labels = ['enet','enet2','enet_kptpl']

nthetabins = 20
thetamax = 250./60.
thetamin = 2.5/60.
bin_slop = 0.05
num_threads = 5

angle_edges = np.logspace(np.log10(thetamin*60.),np.log10(thetamin*60.), 21)


In [5]:
def load_galmap_w(ibin, imock, label):
    mock = lsssys.Mock(None, 512, empty=True )
    mock.load( mock_file_temp.format(imock=imock) )
    galmap_w = mock.ngal[ibin]
    f = lsssys.SysMap(f_map_temp.format(label=label, ibin=ibin, imock=imock) )
    assert (f.data[~galmap_w.mask] == hp.UNSEEN).any() == False
    galmap_w.data[~galmap_w.mask] = galmap_w.data[~galmap_w.mask]/(1. + f.data[~galmap_w.mask]) 
    return galmap_w

In [ ]:
%%capture

#compute clustering
nbins = 1
nmocks = 1

for label in labels:
    for imock in range(nmocks):
        w_dict = {}
        for ibin in range(5):
            try:
                galmap_w = load_galmap_w(ibin, imock, label)
            except IOError:
                print('couldnt find', imock, ibin)
                continue
            
            theta, wtheta = lsssys.corr2pt(galmap_w, galmap_w, 
              nthetabins, thetamax, thetamin, 
              bin_slop=bin_slop, num_threads=num_threads, bin_type=None, 
              delta_input=False, w1=None, w2=None, 
              scale1=1./galmap_w.fracdet, scale2=1./galmap_w.fracdet, 
              return_var=False, 
              returncorr=False, jointmask=None, 
              fracweights=True, fracweights2=True, 
              weights=None, weights2=None)
            w_dict['theta_{0}_{0}'.format(ibin)] = theta
            w_dict[ibin,ibin] = wtheta

        w_dict['angle_min'] = angle_edges[:-1]
        w_dict['angle_max'] = angle_edges[1:]

        spectrum = lsssys.corrdict_2_spectrumtype(w_dict, autoonly=True, name='wtheta', 
                    kernel1='nz_lens', kernel2='nz_lens',)
        
        tp = twopoint.TwoPointFile([spectrum], kernels=None, windows={}, covmat_info=None)
        tp.to_fits('wtheta_redmagic_y3_mock{imock}_bs{bin_slop}_{label}.fits'.format(
            bin_slop = bin_slop,
            label=label, 
            imock=imock), overwrite=True)
        
            